In [1]:
import json
JOB_NAME = 'bean-type-prediction-v2'
PIPELINE = 'gs://zencore-vertex-pipeline-artifacts/compiledpipelines/bean-type-prediction-pipeline.json'
MODEL = 'gs://zencore-vertex-models/beans1'
SPARK_JOB = 'gs://zencore-vertex-pipeline-artifacts/sparkjobs/preprocess.py'
INPUT_TABLE = 'sara-vertex-demos:beans_demo.large_dataset'
OUTPUT_SCHEMA = json.load(open('./schema.json', 'r'))

PROJECT_ID = 'zencore-vertex'
REGION = 'us-east1'

In [4]:
from google.cloud import aiplatform

pipeline_job = aiplatform.PipelineJob(
    project=PROJECT_ID,
    location=REGION,
    display_name=JOB_NAME,
    template_path=PIPELINE,
    pipeline_root=f"gs://{PROJECT_ID}-pipeline-artifacts",
    parameter_values={
        "project_id": PROJECT_ID,
        "location": REGION,
        "display_name": JOB_NAME,
        "dataproc_cluster": "dataproc",
        "dataproc_python_file": SPARK_JOB,
        "input_table": INPUT_TABLE,
        "preprocess_args": ["--sample-fraction", "1.0"],
        "model_uri": MODEL,
        "staging_bucket": f"gs://{PROJECT_ID}-pipeline-artifacts",
        "load_args": ["--schema", json.dumps(OUTPUT_SCHEMA)],
        "output_destination": f"{PROJECT_ID}:vertex.prediction"
    }
)

In [5]:
pipeline_job.submit(service_account=f"vertex-pipeline@{PROJECT_ID}.iam.gserviceaccount.com")

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/576839278996/locations/us-east1/pipelineJobs/scikit-batch-prediction-20211115102011
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/576839278996/locations/us-east1/pipelineJobs/scikit-batch-prediction-20211115102011')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-east1/pipelines/runs/scikit-batch-prediction-20211115102011?project=576839278996


In [6]:
pipeline_job.state

<PipelineState.PIPELINE_STATE_RUNNING: 3>